In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


c:/Users/Admin/Data/ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [2]:
save_dir=f"{exps_dir}/exp2"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(19430, 34) (19430, 1) (5089, 34) (5089, 1)


### (2) Lựa chọn các đặc trưng

In [4]:
x_train

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,week_claimed,week_incident
0,9,2,0,13,1,1,9,3,1,1,-0.507420,0,2,1,0,0.494573,0,1,2,3,2,1,5,4,0,0,0,0,3,0,1,2,559,698
1,5,2,2,6,1,1,5,2,1,1,-0.433660,0,2,1,5,-1.457323,13,1,3,3,2,1,7,4,0,0,0,3,3,0,0,2,140,177
2,8,1,6,13,1,6,6,2,1,1,-0.359901,1,1,0,1,0.977001,13,1,0,3,2,0,5,4,0,0,0,0,3,0,2,1,778,1053
3,8,0,6,18,1,6,8,0,0,1,0.598971,0,2,1,1,1.472873,6,1,3,3,2,2,6,6,0,0,0,0,3,0,2,2,817,1046
4,3,1,5,9,1,5,3,1,0,1,-0.064863,0,2,1,1,0.989773,4,1,1,3,2,1,2,5,0,0,0,0,3,2,2,2,699,891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19425,11,0,0,14,1,1,11,0,0,2,0.103566,0,0,0,2,-0.362216,9,1,2,3,2,3,3,4,0,0,0,0,3,0,0,0,288,359
19426,9,1,2,12,1,3,9,2,1,1,-0.061830,1,0,0,2,-1.023476,2,2,2,3,2,1,5,4,0,0,0,2,1,0,0,0,260,319
19427,2,1,4,9,1,4,2,1,0,1,-0.195377,0,1,0,0,0.187018,5,1,1,3,2,1,5,4,0,0,0,3,3,0,1,1,376,467
19428,3,2,5,13,1,3,3,2,1,1,0.005318,0,3,1,1,0.540914,11,1,0,3,2,1,5,5,0,0,0,2,3,0,1,0,403,508


In [5]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [6]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo Empty DataFrame
Columns: [Feature, Coefficient]
Index: []
Xếp hạng các đặc trưng theo mức độ quan trọng:
                 Feature  Coefficient
31            BasePolicy    -1.155306
11                 Fault    -0.794174
19  Days_Policy_Accident     0.713594
13       VehicleCategory    -0.471186
20     Days_Policy_Claim     0.465878
12            PolicyType    -0.343953
9          MaritalStatus    -0.259468
28   AddressChange_Claim     0.235504
18          DriverRating    -0.234901
15          PolicyNumber    -0.231116
21    PastNumberOfClaims    -0.224736
4           AccidentArea    -0.213569
10                   Age    -0.199792
27   NumberOfSuppliments    -0.175733
22          AgeOfVehicle    -0.168932
17            Deductible     0.148435
29          NumberOfCars    -0.146037
1            WeekOfMonth    -0.138608
7     WeekOfMonthClaimed    -0.100205
5       DayOfWeekClaimed    -0.079602
24     PoliceReportFiled    -0.070842
3                   Make    -0.057695
8      

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [7]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [8]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
models.append(RandomForestClassifier())
feature_RFE={}

names=['LR','LDA','DTC','RD']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['Age', 'AgeOfPolicyHolder', 'AgentType', 'BasePolicy', 'Fault', 'MaritalStatus', 'PoliceReportFiled', 'PolicyType', 'VehicleCategory', 'WitnessPresent']
RFE
Kiểm tra mô hình LDA
['AccidentArea', 'Age', 'AgeOfPolicyHolder', 'BasePolicy', 'Days_Policy_Claim', 'Fault', 'MaritalStatus', 'NumberOfCars', 'PoliceReportFiled', 'VehicleCategory']
RFE
Kiểm tra mô hình DTC
['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PastNumberOfClaims', 'PolicyNumber', 'RepNumber', 'week_claimed', 'week_incident']
RFE
Kiểm tra mô hình RD
['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PolicyNumber', 'PolicyType', 'VehicleCategory', 'week_claimed', 'week_incident']


### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [9]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung GNB
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung NN
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung BG
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], dtype='object')
Lua chon dac trung NB
Index(['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [10]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'GNB': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'NN': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'BG': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'NB': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'SVM': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatu

In [11]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['Age',
  'AgeOfPolicyHolder',
  'AgentType',
  'BasePolicy',
  'Fault',
  'MaritalStatus',
  'PoliceReportFiled',
  'PolicyType',
  'VehicleCategory',
  'WitnessPresent'],
 'LDA': ['AccidentArea',
  'Age',
  'AgeOfPolicyHolder',
  'BasePolicy',
  'Days_Policy_Claim',
  'Fault',
  'MaritalStatus',
  'NumberOfCars',
  'PoliceReportFiled',
  'VehicleCategory'],
 'DTC': ['Age',
  'AgeOfVehicle',
  'BasePolicy',
  'Fault',
  'Make',
  'PastNumberOfClaims',
  'PolicyNumber',
  'RepNumber',
  'week_claimed',
  'week_incident'],
 'RD': ['Age',
  'AgeOfVehicle',
  'BasePolicy',
  'Fault',
  'Make',
  'PolicyNumber',
  'PolicyType',
  'VehicleCategory',
  'week_claimed',
  'week_incident'],
 'KNC': ['DayOfWeek',
  'AccidentArea',
  'WeekOfMonthClaimed',
  'MaritalStatus',
  'Fault',
  'PolicyType',
  'VehicleCategory',
  'DriverRating',
  'AgeOfVehicle',
  'AgeOfPolicyHolder',
  'BasePolicy'],
 'GNB': ['DayOfWeek',
  'AccidentArea',
  'WeekOfMonthClaimed',
  'MaritalStatus',
  'Fault',
 

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [12]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['Age', 'AgeOfPolicyHolder', 'AgentType', 'BasePolicy', 'Fault', 'MaritalStatus', 'PoliceReportFiled', 'PolicyType', 'VehicleCategory', 'WitnessPresent'], 'LDA': ['AccidentArea', 'Age', 'AgeOfPolicyHolder', 'BasePolicy', 'Days_Policy_Claim', 'Fault', 'MaritalStatus', 'NumberOfCars', 'PoliceReportFiled', 'VehicleCategory'], 'DTC': ['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PastNumberOfClaims', 'PolicyNumber', 'RepNumber', 'week_claimed', 'week_incident'], 'RD': ['Age', 'AgeOfVehicle', 'BasePolicy', 'Fault', 'Make', 'PolicyNumber', 'PolicyType', 'VehicleCategory', 'week_claimed', 'week_incident'], 'KNC': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'], 'GNB': ['DayOfWeek', 'AccidentArea', 'WeekOfMonthClaimed', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'DriverRating', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'BasePolicy'],